In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pylab as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import gc 
import time
import optuna
import sklearn.metrics
from datetime import datetime 

plt.style.use('ggplot') # Lets make our plots pretty

path = os.getcwd()

n=datetime.now().strftime("%m_%d_%H_%M")
print(str(n))
print(path)  

02_15_19_45
/home/tomita/kaggle/kaggle_elo/Models


In [2]:
# Read in the dataframes
train = pd.read_csv('../input/train_1.csv')
test = pd.read_csv('../input/test_1.csv')

#print(train.columns)

target = train['target']
train_true=np.array(train['target'])
print(train_true.shape)
                    
del train['target']
del train['outliers']
#del train['outliners']
features = [c for c in train.columns if c not in ['card_id', 'first_active_month']]
categorical_feats = [c for c in features if 'feature_' in c]

(201917,)


In [ ]:
def kfold_lightgbm(trial):
    FEATS_EXCLUDED = ['first_active_month', 'target', 'card_id', 'outliers',
                  'hist_purchase_date_max', 'hist_purchase_date_min', 'hist_card_id_size',
                  'new_purchase_date_max', 'new_purchase_date_min', 'new_card_id_size',
                  'OOF_PRED', 'month_0','outliers']
    seed=20190208
        
    # params optimized by optuna
    learning_rate_tuna = trial.suggest_uniform('learning_rate', 0, 1.0)
    toprate_tuna = trial.suggest_uniform('top_rate', 0, 1.0)
    num_leaves_tuna = trial.suggest_uniform('learning_rate', 0, 1.0)
    min_child_weight_tuna = trial.suggest_int('min_child_weight', 5, 500)
    other_rate_tuna=trial.suggest_uniform('other_rate', 0.0, 1.0)
    num_leaves_tuna=trial.suggest_int('num_leaves', 5, 1000)
    min_gain_split_tuna=trial.suggest_uniform('min_gain_split', 5, 500)
    reg_lambda_tuna=trial.suggest_uniform('reg_lambda', 5, 500)
    subsample_tuna = trial.suggest_uniform('sub_sample', 0, 1.0)
    reg_alpha_tuna=trial.suggest_uniform('sub_sample', 0, 20)
    colsample_bytree_tuna = trial.suggest_uniform('colsample_bytree_tuna', 0, 1.0)
    max_depth= trial.suggest_int('max_depth', 5, 100)
    
    param ={'task': 'train',
            'boosting': 'dart',
            'objective': 'regression',
            'metric': 'rmse',
            'learning_rate': learning_rate_tuna ,
            'subsample': subsample_tuna,
            'max_depth': max_depth,
            'top_rate': toprate_tuna ,
            'num_leaves': num_leaves_tuna,
            'min_child_weight': min_child_weight_tuna,
            'other_rate': other_rate_tuna,
            'reg_alpha': reg_alpha_tuna,
            'colsample_bytree':colsample_bytree_tuna  ,
            'min_split_gain': min_gain_split_tuna,
            'reg_lambda': reg_lambda_tuna,
            'min_data_in_leaf': 21,
            'verbose': -1,
            'seed':seed,
            'bagging_seed':seed,
            'drop_seed':seed,
            'max_bin':255,
            'device':'gpu'
            }

    # Create arrays and dataframes to store results
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    oof = np.zeros(len(train))
    predictions = np.zeros(len(test))
    train_prdictions = np.zeros(train.shape[0])
    start = time.time()
    feature_importance_df = pd.DataFrame()
    
    # k-fold
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
        print("fold n°{}".format(fold_))
        trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=target.iloc[trn_idx], categorical_feature=categorical_feats)
        val_data = lgb.Dataset(train.iloc[val_idx][features], label=target.iloc[val_idx], categorical_feature=categorical_feats)

        num_round = 10000
        clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
        oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = features
        fold_importance_df["importance"] = clf.feature_importance()
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
        train_prdictions += clf.predict(train[features], num_iteration=clf.best_iteration) / folds.n_splits
        predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits

    n=datetime.now().strftime("%m_%d_%H_%M_%S")
    feature_importance_df.to_csv("../output/feature_importance.csv")
    sub_df = pd.read_csv("../input/sample_submission.csv",engine='python')
    sub_df["target"] = predictions
    sub_df.to_csv("../output/submit_lgb"+str(n)+"_optuna.csv", index=False)
    error_train = sklearn.metrics.mean_absolute_error(train_true,train_prdictions)

    return error_train 


In [ ]:
study = optuna.create_study()
study.optimize(kfold_lightgbm, n_trials=150)

fold n°0


/home/tomita/.pyenv/versions/anaconda3-5.3.1/envs/py36/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/tomita/.pyenv/versions/anaconda3-5.3.1/envs/py36/lib/python3.6/site-packages/lightgbm/basic.py:752: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.64055	valid_1's rmse: 3.69642
[200]	training's rmse: 3.63904	valid_1's rmse: 3.69647
[300]	training's rmse: 3.63832	valid_1's rmse: 3.69576
[400]	training's rmse: 3.63792	valid_1's rmse: 3.69546
[500]	training's rmse: 3.63579	valid_1's rmse: 3.69447
[600]	training's rmse: 3.63604	valid_1's rmse: 3.69506
[700]	training's rmse: 3.6354	valid_1's rmse: 3.69447
[800]	training's rmse: 3.63619	valid_1's rmse: 3.69484
[900]	training's rmse: 3.63511	valid_1's rmse: 3.69357
[1000]	training's rmse: 3.63573	valid_1's rmse: 3.69418
[1100]	training's rmse: 3.63608	valid_1's rmse: 3.69437
[1200]	training's rmse: 3.63492	valid_1's rmse: 3.69238
Early stopping, best iteration is:
[1008]	training's rmse: 3.63403	valid_1's rmse: 3.69287
fold n°1
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.65866	valid_1's rmse: 3.63037
[200]	training's rmse: 3.65331	valid_1's rmse: 3.62731
[3

[I 2019-02-15 22:14:36,953] Finished a trial resulted in value: 1.5524799357442964. Current best value is 1.5524799357442964 with parameters: {'learning_rate': 0.6129707926958924, 'top_rate': 0.6351386486354692, 'min_child_weight': 271, 'other_rate': 0.6996507555220448, 'num_leaves': 476, 'min_gain_split': 366.56499230255895, 'reg_lambda': 222.92932996562237, 'sub_sample': 0.08474710317963396, 'colsample_bytree_tuna': 0.9520231859522077, 'max_depth': 19}.


fold n°0
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.22484	valid_1's rmse: 3.71035
[200]	training's rmse: 3.2319	valid_1's rmse: 3.69983
[300]	training's rmse: 3.24343	valid_1's rmse: 3.69469
Early stopping, best iteration is:
[117]	training's rmse: 3.20886	valid_1's rmse: 3.70984
fold n°1
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.23056	valid_1's rmse: 3.63265
[200]	training's rmse: 3.24715	valid_1's rmse: 3.62315
Early stopping, best iteration is:
[98]	training's rmse: 3.2259	valid_1's rmse: 3.63392
fold n°2
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.24867	valid_1's rmse: 3.62312
[200]	training's rmse: 3.2729	valid_1's rmse: 3.6127
Early stopping, best iteration is:
[98]	training's rmse: 3.24276	valid_1's rmse: 3.62634
fold n°3
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.22306	valid_1's rmse: 3.81066
[200]	tr

[I 2019-02-16 02:17:53,825] Finished a trial resulted in value: 1.4503054340278518. Current best value is 1.4503054340278518 with parameters: {'learning_rate': 0.6467720115100732, 'top_rate': 0.7790255922489042, 'min_child_weight': 85, 'other_rate': 0.05539729354616829, 'num_leaves': 987, 'min_gain_split': 41.836633393708205, 'reg_lambda': 246.1691414168485, 'sub_sample': 0.4087483998244248, 'colsample_bytree_tuna': 0.5331266981851617, 'max_depth': 57}.


fold n°0
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.49874	valid_1's rmse: 3.68305
[200]	training's rmse: 3.5026	valid_1's rmse: 3.67887
[300]	training's rmse: 3.49458	valid_1's rmse: 3.67675
[400]	training's rmse: 3.49169	valid_1's rmse: 3.6776
Early stopping, best iteration is:
[291]	training's rmse: 3.49449	valid_1's rmse: 3.6765
fold n°1
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.50797	valid_1's rmse: 3.61887
[200]	training's rmse: 3.50709	valid_1's rmse: 3.61845
[300]	training's rmse: 3.50396	valid_1's rmse: 3.61871
Early stopping, best iteration is:
[172]	training's rmse: 3.5069	valid_1's rmse: 3.61581
fold n°2
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.52014	valid_1's rmse: 3.60555
[200]	training's rmse: 3.51414	valid_1's rmse: 3.60411
[300]	training's rmse: 3.51538	valid_1's rmse: 3.60137
[400]	training's rmse: 3.5117	valid_1's rmse: 3.60066
[500

In [ ]:
feature_importance_df=pd.read_csv("../output/feature_importance.csv")
cols = (feature_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:20].index)

best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')

In [8]:
study.best_params

{'colsample_bytree_tuna': 0.83435723889734326,
 'learning_rate': 0.75836372582243783,
 'max_depth': 29,
 'min_child_weight': 27,
 'min_gain_split': 27.378180277455101,
 'num_leaves': 562,
 'other_rate': 0.03564199289369395,
 'reg_lambda': 94.549009291544877,
 'sub_sample': 0.91158142068248083,
 'top_rate': 0.72876375065913579}

In [ ]:
n=10
sub_df = pd.read_csv("../input/sample_submission.csv")
sub_df["target"] = predictions
sub_df.to_csv("../output/submit_lgb"+str(n)+"_optuna.csv", index=False)